### Import Relevant Packages

In [19]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Load dataset


In [20]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

# We choose the validation dataset to be 10% of the training dataset
num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# Store number of test samples in a variable
num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Define function to scale data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.float32)
    image /= 255.

    return image, label

scaled_train_val_dataset = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# Shuffle the data
BUFFER_SIZE = 10000

shuffled_train_val_dataset = scaled_train_val_dataset.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_val_dataset.take(num_validation_samples)
train_data = shuffled_train_val_dataset.skip(num_validation_samples)

# Batch the data for GD
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

### Outline The Model

In [21]:
# We start with 2 hidden layers of width 50
input_size = 784
hidden_layer_size = 100
output_size = 10

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

### Select loss function and optimizer

In [22]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

### Training


In [23]:
from tabnanny import verbose

NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 6s - loss: 0.4089 - accuracy: 0.8881 - val_loss: 0.2130 - val_accuracy: 0.9378 - 6s/epoch - 11ms/step
Epoch 2/5
540/540 - 3s - loss: 0.1907 - accuracy: 0.9452 - val_loss: 0.1653 - val_accuracy: 0.9512 - 3s/epoch - 6ms/step
Epoch 3/5
540/540 - 4s - loss: 0.1451 - accuracy: 0.9578 - val_loss: 0.1313 - val_accuracy: 0.9603 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 4s - loss: 0.1199 - accuracy: 0.9643 - val_loss: 0.1164 - val_accuracy: 0.9648 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 3s - loss: 0.1013 - accuracy: 0.9702 - val_loss: 0.0988 - val_accuracy: 0.9692 - 3s/epoch - 6ms/step


### Testing


In [24]:
test_loss, test_accuracy = model.evaluate(test_data)
print("Test loss: {:.2f}\nTest accuracy: {:.2f}".format(test_loss, test_accuracy))

1/1 [==============================] - 1s 1s/step - loss: 0.1119 - accuracy: 0.9649
Test loss: 0.11
Test accuracy: 0.96
